In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time # 사이트 기동될 때 까지 2초 정도를 줌. - 셀리니움에서 get. request에서 get쓸 때
import pymysql

In [2]:
def chosunnewsUrl(search):
        url_temp = 'https://nsearch.chosun.com/search/total.search?query={search}&sort=1'.format(search=search)
        driver = webdriver.Chrome("./chromedriver.exe")
        driver.get(url_temp)
        time.sleep(2)
        
        return chosunnewsUrlList(driver)

In [1]:
def chosunnewsUrlList(driver):
    url_temp = driver.current_url 
    # 메모리의 위치값에서 current_url -> 크롬 브라우저의 주소값을 넘겨줌
    for pageNo in range(1, 3, 1):
        url = url_temp + "&pageNum={pageNo}".format(pageNo=pageNo)
        print(url)
        driver.get(url)
        time.sleep(2)
        
        chosunnewsUrls_List = []
        chosunnewsUrls = driver.find_elements_by_css_selector(".search_news_box .search_news .thumb a")
        
        # 중간에 배너광고를 배제시키기 위해서 사용.
        for urlList in chosunnewsUrls:
            if urlList.get_attribute("href")[8:12] == 'news':
                chosunnewsUrls_List.append(urlList.get_attribute("href"))
            else:
                pass
        return chosunnewsInfo(driver, chosunnewsUrls_List)

In [2]:
def chosunnewsInfo(driver, chosunnewsUrls_List):
    for urllist in chosunnewsUrls_List:
        driver.get(urllist)
        time.sleep(2)
        # 하나씩 가져와서 뛰우기
        print(urllist)
        #newsTitle은 list
        
#         newsTitleList = driver.find_element_by_css_selector("#news_title_text_id").text
        #.text는 태그는 안오고 텍스트만 가져옴.
        #s가 안붙여서 스트링 하나. 
#         # 같은 거
#         newsTitleList = driver.find_elements_by_css_selector("#news_title_text_id")
#         newsTitle = newsTitleList[0].text

        # 내가 몇 개 나오든 상관없이 똑같이 쓸 수 있는데 위는 재활용측면에서 선호하지 않음
        newsTitleList = driver.find_elements_by_css_selector("#news_title_text_id")
        newsTitle_temp = newsTitleList[0]
        newsTitle = newsTitle_temp.text
        print(newsTitle)

        # 날짜
        newsDate = ""
        newsDateList = driver.find_elements_by_css_selector(".news_date")
        newsDate_temp = newsDateList[0]
        newsDate = newsDate_temp.text
        newsDate = newsDate.replace("입력 ", "").replace(".", "")[0:8]
    
        # 서브타이틀
        newsSubtitle = ""
        newssubTitleList = driver.find_elements_by_css_selector(".news_subtitle")
        newssubTitle_temp = newssubTitleList[0]
        newssubTitle = newssubTitle_temp.text
        print(newssubTitle)
#         print(newsTitle)
    
        newsContents = ""
        newsContentsList = driver.find_elements_by_css_selector("#news_body_id .par")
        for newsContent in newsContentsList:
            newsContents = newsContents + newsContent.text
        print(newsContents)
        
        dbData = [[urllist, newsDate, newsTitle, newsSubtitle, newsContents]]
        connectDB(dbData)

In [3]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'python'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into chosun(url, newsDate, newsTitle,newsSubtitle,contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [7]:
search = '남북관계'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
chosunnewsUrl(search)

https://nsearch.chosun.com/search/total.search?query=%EB%82%A8%EB%B6%81%EA%B4%80%EA%B3%84&sort=1&pn=1
https://news.chosun.com/site/data/html_dir/2020/05/07/2020050702119.html
김연철 통일 "판문점 견학, 6월부터 시범 재개"
GP 총격 직후인데…"안전 문제는 다각적으로 보완"
"판문점, 중부전선과 달라…北에도 중요 관광자원"
지난해 10월 아프리카돼지열병(ASF) 방역 조치로 중단된 판문점 견학이 다음달부터 시범 관광 형식으로 재개될 예정이다. 김연철 통일부 장관은 7일 출입기자단 간담회에서 “5월 중순 환경부에서 하는 멧돼지 검체 채취 조사를 최종판단의 근거로 삼아 6월부터 소수의 인원으로 시범관광 준비를 하고 있다”고 말했다.

앞서 정부는 지난해 가을 접경지역에 ASF가 급속히 확산하자 판문점 견학과 ‘DMZ 평화의 길’ 개방을 중단했다. 김 장관은 판문점 견학 재개와 관련, “안전 문제는 다각적으로 보완을 했다”고 밝혔다. 김 장관은 “판문점은 (최근 총격사건이 발생한) 중부전선하고는 많이 다르다”며 “판문점은 전통적으로 북한 입장에서도 매우 중요한 관광자원”이라고 말했다.

북한군이 DMZ(비무장지대) 내 우리 군 최전방 감시소초(GP)에 총격을 가한 지 나흘 만에 정부 고위 인사가 최전방 지역에서의 민간인 관광 재개 방침을 구체적으로 밝힌 것이다. 김 장관은 전날 판문점과 ‘DMZ 평화의 길’ 파주 구간을 시찰하며 견학·탐방 재개 준비 상황을 살폈다.

이날 김 장관은 GP 총격 사건에 대해서는 언급을 자제했다. 김 장관은 “(총격 사건은) 국방부서 상세한 설명이 있었던 것으로 안다. 전후 맥락, 상황 등을 충분히 설명한 것으로 알고 있다”며 “유엔사와 군정위도 그 부분은 조사를 하고 있고 조만간 입장이 발표될 것으로 안다”고 했다.

김 장관은 코로나 사태로 진척을 보지 못한 남북 협력사업들을 속도감 있게 추진하